In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [ ]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow-hub transformers


In [1]:
import nltk
import subprocess
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/content/')
    command = "unzip /content/corpora/wordnet.zip -d /content/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/content/')


[nltk_data] Downloading package wordnet to /content/...


In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
!pip install pandarallel vaderSentiment

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.7 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=a6fad4341cfcb3eddbd5a7dbc767490f959ffbd0ec0b715a236c0a7f9b620905
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [5]:
import pandas as pd
import re
chat_corpus = pd.read_csv('/content/dialogueText_196 (1).csv')
chat_corpus.dropna(subset=['text'], inplace=True)
def atleast_one_en_char_flag(text):
    return not (re.search("[a-zA-Z]", text)==None)

chat_corpus = chat_corpus[chat_corpus.text.apply(atleast_one_en_char_flag)]



chat_corpus.info()


<class 'pandas.core.frame.DataFrame'>
Index: 302697 entries, 0 to 305077
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   folder      302697 non-null  int64 
 1   dialogueID  302697 non-null  object
 2   date        302697 non-null  object
 3   from        302697 non-null  object
 4   to          240554 non-null  object
 5   text        302697 non-null  object
dtypes: int64(1), object(5)
memory usage: 16.2+ MB


In [ ]:
len(chat_corpus)

48545

In [ ]:
chat_corpus

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...
...,...,...,...,...,...,...
48919,196,1.tsv,2011-01-01T21:06:00.000Z,juniour,rs0832,happy new year
48920,196,1.tsv,2011-01-01T21:07:00.000Z,rs0832,juniour,happy new year to you too :)
48921,196,1.tsv,2011-01-02T20:20:00.000Z,juniour,rs0832,hi
48922,196,1.tsv,2011-01-02T20:21:00.000Z,rs0832,juniour,hi... im good .. you?


In [6]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

chat_corpus = chat_corpus[['dialogueID', 'from', 'to', 'text']]
chat_corpus['to'] = chat_corpus['to'].fillna('')

inputs = chat_corpus['text'][:-1].reset_index(drop=True)
outputs = chat_corpus['text'][1:].reset_index(drop=True)

dialogue_ids = chat_corpus['dialogueID'][:-1].reset_index(drop=True)
chat_data = pd.DataFrame({'dialogueID': dialogue_ids, 'input': inputs, 'output': outputs})

chat_data.head()


<ipython-input-6-4ca1a446a4d4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chat_corpus['to'] = chat_corpus['to'].fillna('')


,dialogueID,input,output
0,1.tsv,any ideas why java plugin takes so long to load?,java 1.4?
1,1.tsv,java 1.4?,yes
2,1.tsv,yes,java 1.5 loads _much_ faster
3,1.tsv,java 1.5 loads _much_ faster,noneus: how can i get 1.5 is there a .deb some...
4,1.tsv,noneus: how can i get 1.5 is there a .deb some...,not yet.



1. **Setup**:
   - A copy of `chat_data` is stored in `df`, with new columns `new_input` and `new_output` initialized to empty strings.

2. **Step 1 - Create Input History**:
   - Starting from the 6th row (index 5), the code generates a history of the previous five messages within the same `dialogueID` and stores it in `new_input`.
   - It checks if the `dialogueID` of the current row matches that of the row five steps earlier. If they match:
     - The history string is formed by concatenating the previous five `input` messages from `i-5` to `i-1` with "[FS]" separators, followed by the last output (`df.loc[i-1, 'output']`).
     - This history string is stored in `new_input` for the current row `i`.

3. **Step 2 - Append Previous Output to Input**:
   - Starting from the 7th row (index 6), the code appends the `new_output` from the previous row to `new_input` in the current row if they share the same `dialogueID`.
   - This step extends the history in `new_input` with each previous row’s output.

4. **Final Step - Set `new_output`**:
   - The `new_output` column is filled with the current row's original `output`, with "[FS]" appended at the end.

This code essentially constructs a sequential input history for each message, combining the past five inputs and previous outputs where relevant, enabling the context to "carry over" throughout a dialogue.

In [7]:
import pandas as pd

df = chat_data.copy()
df['new_input'] = ''
df['new_output'] = ''

# Step 1: Create the input history (the previous 5 inputs) within the same dialogueID
for i in range(5, len(df)):
    if df.loc[i, 'dialogueID'] == df.loc[i-5, 'dialogueID']:
        history = " [FS] ".join(df.loc[i-5:i-1, 'input'].astype(str)) + ' [FS] ' + df.loc[i-1, 'output'] + ' [FS]'
        df.loc[i, 'new_input'] = history

# Step 2: Now append the previous row's output to the current row's input (after step 1 is done)
for i in range(6, len(df)):  # Start from the 7th row, since the 6th row is the first with 5 history rows
    if df.loc[i, 'dialogueID'] == df.loc[i-1, 'dialogueID']:
        df.loc[i, 'new_input'] += " [FS] " + df.loc[i-1, 'new_output']

# Set the new output as the current row's output and append [FS] at the end of each message
df['new_output'] = df['output'] + ' [FS]'

print(df[['new_input', 'new_output']].head(10))


                                           new_input  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      
5  any ideas why java plugin takes so long to loa...   
6  java 1.4? [FS] yes [FS] java 1.5 loads _much_ ...   
7  yes [FS] java 1.5 loads _much_ faster [FS] non...   
8  java 1.5 loads _much_ faster [FS] noneus: how ...   
9  noneus: how can i get 1.5 is there a .deb some...   

                                          new_output  
0                                     java 1.4? [FS]  
1                                           yes [FS]  
2                  java 1.5 loads _much_ faster [FS]  
3  noneus: how can i get 1.5 is there a .deb some...  
4                                      not yet. [FS]  
5             noneus: is this blackdown or sun? [FS] 

In [ ]:
df['new_input'][5]

'any ideas why java plugin takes so long to load? [FS] java 1.4? [FS] yes [FS] java 1.5 loads _much_ faster [FS] noneus: how can i get 1.5 is there a .deb somewhere? [FS] not yet. [FS]'

In [ ]:
df['new_output'][5]

'noneus: is this blackdown or sun? [FS]'

In [ ]:
df['new_input'][6]

'java 1.4? [FS] yes [FS] java 1.5 loads _much_ faster [FS] noneus: how can i get 1.5 is there a .deb somewhere? [FS] not yet. [FS] noneus: is this blackdown or sun? [FS] [FS] '

In [ ]:
first_row = df.loc[df['dialogueID'] == '10.tsv'].head(1)
first_row


,dialogueID,input,output,new_input,new_output
3758,10.tsv,"hey, I'm playing with window managers. What d...",Don't you put which WM to start in ~/.xinitrc ?,,Don't you put which WM to start in ~/.xinitrc...


In [ ]:
df['dialogueID'][3758]


'10.tsv'

In [ ]:
df['new_input'][3757]


'(!who) sure, but you can use ONE (like a 10 gig) to install win, add a module for ext3 later to win [FS] : i fixed to partitions , thanks very much :D , can u show me topic in any forum about hacking wireless networks , i read that linux is strong on that [FS] !google|kebomix: [FS] tag : wardriving, cu [FS] thanks very much , i still have one small problem [FS] if i have movie in 3 connected rar files , then i have to extract 3 files at once to show me one file and it is the movie , here in ubuntu , every file extract with it self and give me three files instead of one file , and movie dontwork , it dont read here that files is connected !!!!! [FS] [FS] '

In [ ]:
df['new_input'][3763]

"hey, I'm playing with window managers.  What does gdm look at now?  I don't see ~/.xsession files. [FS]  Don't you put which WM to start in ~/.xinitrc ? [FS] I don't seem to have a ~/.xinitrc and I'm using Unity's login manager. [FS]  Sorry I don't know anything about Unity, but just because you don't have ~/.x* doesn't mean they won't work if you create them. The system wide .x* files are in /etc somewhere iirc [FS]  in Youtube Account Preferences you can choose HTML5 video instead of flash [FS]  hey, what is your problem with awesome? maybe I can help; I use it [FS] [FS] "

In [ ]:
df['new_input'][3765]

"I don't seem to have a ~/.xinitrc and I'm using Unity's login manager. [FS]  Sorry I don't know anything about Unity, but just because you don't have ~/.x* doesn't mean they won't work if you create them. The system wide .x* files are in /etc somewhere iirc [FS]  in Youtube Account Preferences you can choose HTML5 video instead of flash [FS]  hey, what is your problem with awesome? maybe I can help; I use it [FS] I can't get it to read my rc.lua in my home directory$XDG_CONFIG_HOME. [FS]  Check what it says in ~/.cache/awesome/stderr [FS] [FS] "

In [ ]:
len(df)

19126

In [ ]:
df

,dialogueID,input,output,new_input,new_output
0,1.tsv,any ideas why java plugin takes so long to load?,java 1.4?,,java 1.4? [FS]
1,1.tsv,java 1.4?,yes,,yes [FS]
2,1.tsv,yes,java 1.5 loads _much_ faster,,java 1.5 loads _much_ faster [FS]
3,1.tsv,java 1.5 loads _much_ faster,noneus: how can i get 1.5 is there a .deb some...,,noneus: how can i get 1.5 is there a .deb some...
4,1.tsv,noneus: how can i get 1.5 is there a .deb some...,not yet.,,not yet. [FS]
...,...,...,...,...,...
19121,41.tsv,i'm assuming your using lucid. then i would sa...,that particular model is supposed to work with...,dell studio 15 i would have to flip it over fo...,that particular model is supposed to work with...
19122,41.tsv,that particular model is supposed to work with...,I downloaded a driver from broadcom that is su...,dell studio 1537 [FS] do you know which partic...,I downloaded a driver from broadcom that is su...
19123,41.tsv,I downloaded a driver from broadcom that is su...,look in restricted drivers and you can see if ...,"do you know which particular broadcom it is, c...",look in restricted drivers and you can see if ...
19124,41.tsv,look in restricted drivers and you can see if ...,how do I do that,I ran a command last night that told me but I ...,how do I do that [FS]


In [ ]:
df = df.drop(columns=['input'])


In [8]:
df = df.drop(columns=['output'])


In [9]:
df = df.iloc[5:]
df

,dialogueID,input,new_input,new_output
5,1.tsv,not yet.,any ideas why java plugin takes so long to loa...,noneus: is this blackdown or sun? [FS]
6,1.tsv,noneus: is this blackdown or sun?,java 1.4? [FS] yes [FS] java 1.5 loads _much_ ...,did you install just the jre? [FS]
7,1.tsv,did you install just the jre?,yes [FS] java 1.5 loads _much_ faster [FS] non...,I use IBM's 1.4.2 [FS]
8,1.tsv,I use IBM's 1.4.2,java 1.5 loads _much_ faster [FS] noneus: how ...,"(jdk, because I do globus development) [FS]"
9,1.tsv,"(jdk, because I do globus development)",noneus: how can i get 1.5 is there a .deb some...,"ah ok then, i'll give it a try, does IBM have ..."
...,...,...,...,...
302691,2285.tsv,changed all the permi9ssion for that folder?,thanks for your patience :) [FS] how do i make...,yes [FS]
302692,2285.tsv,yes,how do i make it readible to apache? [FS] find...,thanks OSOO much for all your help [FS]
302693,2285.tsv,thanks OSOO much for all your help,find -type f -print0 | xargs -o chmod ug+r ; f...,does ubuntu slow down on live usb ? <--- my qu...
302694,129.tsv,does ubuntu slow down on live usb ? <--- my qu...,,erf [FS]


In [10]:
# Remove rows where 'new_input' or 'new_output' columns are only '[FS]'
df = df[~df['new_input'].str.strip().eq('[FS]')]
df = df[~df['new_output'].str.strip().eq('[FS]')]



In [11]:
fraction = 0.1
n_rows = int(len(df) * fraction)
sample_corpus = df.iloc[:n_rows].reset_index(drop=True)


In [12]:
len(sample_corpus)

24942

In [13]:
sample_corpus

,dialogueID,input,new_input,new_output
0,1.tsv,not yet.,any ideas why java plugin takes so long to loa...,noneus: is this blackdown or sun? [FS]
1,1.tsv,noneus: is this blackdown or sun?,java 1.4? [FS] yes [FS] java 1.5 loads _much_ ...,did you install just the jre? [FS]
2,1.tsv,did you install just the jre?,yes [FS] java 1.5 loads _much_ faster [FS] non...,I use IBM's 1.4.2 [FS]
3,1.tsv,I use IBM's 1.4.2,java 1.5 loads _much_ faster [FS] noneus: how ...,"(jdk, because I do globus development) [FS]"
4,1.tsv,"(jdk, because I do globus development)",noneus: how can i get 1.5 is there a .deb some...,"ah ok then, i'll give it a try, does IBM have ..."
...,...,...,...,...
24937,266.tsv,whats that,web search for gparted live cd [FS] can i us...,boot the live cd install testdisk and try it...
24938,266.tsv,boot the live cd install testdisk and try it.,can i use gnome partistion editor [FS] that is...,where do i get test disk [FS]
24939,266.tsv,where do i get test disk,that is gparted run it from a live cd. you...,that code you keep sending me does nothing [FS]
24940,266.tsv,that code you keep sending me does nothing,ok thanks im downloading it and i found that o...,see ubottu's next line... [FS]


In [14]:

class TextPreProcessing:

    def __init__(self, use_lemmatization=True):
        self.CUSTOM_STOPWORDS = ["hi", "try", "want", "thanks", "dont", "know",
                                 "like", "way", "hello", "hey", 'im']
        self.STOPWORDS = self.CUSTOM_STOPWORDS
        self.LEMMATIZER = WordNetLemmatizer()
        self.STEMMER = PorterStemmer()
        self.use_lemmatization = use_lemmatization  # Option to use lemmatization or stemmin
        # Emoticon mapping

        self.EMOTICON_MAP = {
            r":‑\)": " POSITIVE_EMOJI ", r":\)": " POSITIVE_EMOJI ",
            r"=\)": " POSITIVE_EMOJI ", r":3": " POSITIVE_EMOJI ",
            r":\(": " NEGATIVE_EMOJI ", r":‑\(": " NEGATIVE_EMOJI ",
            r":-\/": " NEGATIVE_EMOJI ", r":O": " NEUTRAL_EMOJI ",
            r":-\|": " NEUTRAL_EMOJI "

        }




        self.CONTRACTION_MAP = {

            "don't": "do not", "won't": "will not", "can't": "can not", "i'm": "i am",
            "he's": "he is", "she's": "she is", "that's": "that is", "what's": "what is",
            "where's": "where is", "it's": "it is", "they're": "they are", "i've": "i have",
            "you've": "you have", "we've": "we have", "isn't": "is not", "wasn't": "was not",
            "aren't": "are not", "weren't": "were not", "i'll": "i will", "'ll": " will",
            "'ve": " have", "'re": " are", "'d": " would", "cannot": "can not" , "thx": "thanks"

        }



    def lowercase_text(self, text):
        return text.lower()



    def replace_emojis(self, text):
        for emoji, sentiment_tag in self.EMOTICON_MAP.items():
            text = re.sub(emoji, sentiment_tag, text)
        return text



    def expand_contractions(self, text):
        for contraction, full_form in self.CONTRACTION_MAP.items():
            text = re.sub(contraction, full_form, text)
        return text



    def remove_links(self, text):
        return re.sub(r'http\S+|www\S+', '', text)



    def remove_emails(self, text):
        return re.sub(r'\S+@\S+', '', text)



    def remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', "!\"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~"))



    def remove_double_whitespace(self, text):
        return re.sub(r'\s+', ' ', text).strip()



    def remove_stopwords(self, text):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in self.STOPWORDS]
        return ' '.join(filtered_words)



    def lemmatize(self, text):
        words = word_tokenize(text)
        lemmatized_words = [self.LEMMATIZER.lemmatize(word) for word in words]
        return ' '.join(lemmatized_words)



    def stem(self, text):
        words = word_tokenize(text)
        stemmed_words = [self.STEMMER.stem(word) for word in words]
        return ' '.join(stemmed_words)



    def mask_digits(self, text):
        return re.sub(r'\d+', ' NUMERIC ', text)



    def preprocess_text(self, text, remove_digits=True, remove_emails=True):
        text = self.lowercase_text(text)
        text = self.expand_contractions(text)
        text = self.remove_links(text)

        if remove_emails:
            text = self.remove_emails(text)



        text = self.replace_emojis(text)
        text = self.remove_punctuation(text)
        text = self.remove_double_whitespace(text)
        text = self.remove_stopwords(text)


        if remove_digits:
            text = self.mask_digits(text)


        if self.use_lemmatization:
            text = self.lemmatize(text)

        else:
            text = self.stem(text)


        return text


In [15]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
sample_corpus['new_input'] = sample_corpus['new_input'].parallel_apply(TextPreProcessing().preprocess_text)
sample_corpus['new_output'] = sample_corpus['new_output'].parallel_apply(TextPreProcessing().preprocess_text)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
sample_corpus

,dialogueID,input,new_input,new_output
0,1.tsv,not yet.,any idea why java plugin take so long to load ...,noneus is this blackdown or sun f
1,1.tsv,noneus: is this blackdown or sun?,java NUMERIC . NUMERIC f yes f java NUMERIC . ...,did you install just the jre f
2,1.tsv,did you install just the jre?,yes f java NUMERIC . NUMERIC load much faster ...,i use ibms NUMERIC . NUMERIC . NUMERIC f
3,1.tsv,I use IBM's 1.4.2,java NUMERIC . NUMERIC load much faster f none...,jdk because i do globus development f
4,1.tsv,"(jdk, because I do globus development)",noneus how can i get NUMERIC . NUMERIC is ther...,ah ok then i will give it a doe ibm have a sta...
...,...,...,...,...
24937,266.tsv,whats that,web search for gparted live cd f can i use gno...,boot the live cd install testdisk and it . f
24938,266.tsv,boot the live cd install testdisk and try it.,can i use gnome partistion editor f that is gp...,where do i get test disk f
24939,266.tsv,where do i get test disk,that is gparted run it from a live cd . you ca...,that code you keep sending me doe nothing f
24940,266.tsv,that code you keep sending me does nothing,ok downloading it and i found that out it wont...,see ubottus next line ... f


In [17]:
sample_corpus.to_csv('/content/final_df.csv')



1. **Grouping and Shuffling**:
   - The code groups `sample_corpus` by `dialogueID` (to keep all entries for each dialogue together) and then shuffles each dialogue group using a fixed random state (`42`), which ensures that each dialogueID’s rows remain together but in a randomized order. The result is stored in `shuffled_groups`.

2. **Splitting by `dialogueID`**:
   - Unique `dialogueID` values are extracted from `sample_corpus` to use as a basis for splitting the dataset.
   - Using `train_test_split`, `unique_ids` is divided into:
     - `train_ids` (80% of the dialogueIDs).
     - `temp_ids` (20% of the dialogueIDs, to be split further).

3. **Validation and Test Splits**:
   - The `temp_ids` (20% of dialogueIDs) are split evenly into `val_ids` and `test_ids`, each taking 10% of the original dataset. This results in:
     - 80% of the data in the training set.
     - 10% in the validation set.
     - 10% in the test set.

4. **Creating Subsets**:
   - Three subsets (`train_data`, `val_data`, and `test_data`) are created by selecting rows where the `dialogueID` is within the respective ID list (`train_ids`, `val_ids`, and `test_ids`).

5. **Output**:
   - The number of rows in each subset (training, validation) is printed to confirm the distribution.

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split


grouped = sample_corpus.groupby('dialogueID')

shuffled_groups = grouped.apply(lambda x: x.sample(frac=1, random_state=42)).reset_index(drop=True)

unique_ids = sample_corpus['dialogueID'].unique()
train_ids, temp_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

train_data = sample_corpus[sample_corpus['dialogueID'].isin(train_ids)]
val_data = sample_corpus[sample_corpus['dialogueID'].isin(val_ids)]
test_data = sample_corpus[sample_corpus['dialogueID'].isin(test_ids)]

print(f"Training set: {train_data.shape[0]} rows")
print(f"Validation set: {val_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")


Training set: 19703 rows
Validation set: 2733 rows
Test set: 2506 rows


<ipython-input-18-c2e8320937af>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  shuffled_groups = grouped.apply(lambda x: x.sample(frac=1, random_state=42)).reset_index(drop=True)


app2

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_data['input_tokenized'] = train_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
train_data['output_tokenized'] = train_data['new_output'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

print("train Sample tokenized input:", train_data['input_tokenized'].head())
print("train Sample tokenized output:", train_data['output_tokenized'].head())

val_data['input_tokenized'] = val_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
val_data['output_tokenized'] = val_data['new_output'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

print("val Sample tokenized input:", val_data['input_tokenized'].head())
print("val Sample tokenized output:", val_data['output_tokenized'].head())



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-20-5cee1506e40c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['input_tokenized'] = train_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
<ipython-input-20-5cee1506e40c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

train Sample tokenized input: 291                                           [101, 102]
292    [101, 2003, 2009, 7929, 2000, 5587, 1037, 2240...
293    [101, 2053, 7078, 2025, 1042, 2017, 2064, 2224...
294    [101, 2017, 2064, 2224, 1037, 2067, 6442, 2065...
295    [101, 2054, 2003, 2009, 1042, 4224, 23356, 212...
Name: input_tokenized, dtype: object
train Sample tokenized output: 291                   [101, 2053, 7078, 2025, 1042, 102]
292    [101, 2092, 2017, 2228, 3308, 2067, 25378, 200...
293    [101, 2065, 2017, 3524, 1037, 2096, 1045, 2097...
294    [101, 2681, 1012, 9530, 8873, 27390, 2063, 289...
295    [101, 2017, 2064, 16500, 1996, 20014, 2890, 23...
Name: output_tokenized, dtype: object


<ipython-input-20-5cee1506e40c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['input_tokenized'] = val_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


val Sample tokenized input: 0    [101, 2151, 2801, 2339, 9262, 13354, 2378, 220...
1    [101, 9262, 16371, 25531, 1012, 16371, 25531, ...
2    [101, 2748, 1042, 9262, 16371, 25531, 1012, 16...
3    [101, 9262, 16371, 25531, 1012, 16371, 25531, ...
4    [101, 3904, 2271, 2129, 2064, 1045, 2131, 1637...
Name: input_tokenized, dtype: object
val Sample tokenized output: 0    [101, 3904, 2271, 2003, 2023, 2304, 7698, 2030...
1    [101, 2106, 2017, 16500, 2074, 1996, 3781, 206...
2    [101, 1045, 2224, 9980, 2015, 16371, 25531, 10...
3    [101, 26219, 2243, 2138, 1045, 2079, 1043, 413...
4    [101, 6289, 7929, 2059, 1045, 2097, 2507, 2009...
Name: output_tokenized, dtype: object


<ipython-input-20-5cee1506e40c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['output_tokenized'] = val_data['new_output'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [21]:
vocab_size = 30000
max_len = 100

train_input_sequences = pad_sequences(train_data['input_tokenized'], maxlen=max_len, padding='post', truncating='post')
train_output_sequences = pad_sequences(train_data['output_tokenized'], maxlen=max_len, padding='post', truncating='post')

print("train Sample tokenized input (padded):", train_input_sequences[:5])
print("train Sample tokenized output (padded):", train_output_sequences[:5])

val_input_sequences = pad_sequences(val_data['input_tokenized'], maxlen=max_len, padding='post', truncating='post')
val_output_sequences = pad_sequences(val_data['output_tokenized'], maxlen=max_len, padding='post', truncating='post')

print("val Sample tokenized input (padded):", val_input_sequences[:5])
print("val Sample tokenized output (padded):", val_input_sequences[:5])


train Sample tokenized input (padded): [[  101   102     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  101  2003  2009  7929  2000  5587  1037 22409  2008  2003 20014  2890
  23267  2000  9532  1042  2053  7078  2025  1042  2017  2064  2224  1037
   2067  6442  2065  2045  2003  2028  2030  2017  2064 21953  1996  3120
   7427  1997 20014  2890 23267  1042  2054  2003  2009  1042  4224 23356
   2121 16371 25531  1012 16371 25531  1012 16

In [22]:
import numpy as np

print(np.isnan(train_input_sequences).any())
print(np.isnan(train_output_sequences).any())


False
False


In [23]:
print("Min and Max in train_output_sequences:", train_output_sequences.min(), ",", train_output_sequences.max())


Min and Max in train_output_sequences: 0 , 29656


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def transformer_model(vocab_size, max_len, embed_dim=128, num_heads=8, ff_dim=512, num_transformer_blocks=4, rate=0.1):
    inputs = Input(shape=(max_len,))
    embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
    x = embedding_layer(inputs)

    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate)(x)

    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(rate)(x)
    outputs = Dense(vocab_size, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


model = transformer_model(vocab_size=vocab_size, max_len=max_len)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.build(input_shape=(None, max_len))
model.summary()


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)          │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_2       │ (None, 100, 128)            │       3,852,800 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_8                  │ (None, 100, 128)            │         659,712 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_9                  │ (None, 100, 128)            │         659,712 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_10                 │ (None, 100, 128)            │         659,712 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_11                 │ (None, 100, 128)            │         659,712 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 100, 512)            │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 100, 30000)          │      15,390,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,947,696 (83.72 MB)

 Trainable params: 21,947,696 (83.72 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.001)
history = model.fit(
    train_input_sequences,
    train_output_sequences,
    epochs=100,
    batch_size=32,
    validation_data=(val_input_sequences, val_output_sequences),
    callbacks=[reduce_lr]
)

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 87s 435ms/step - accuracy: 0.7119 - loss: 8.3111 - val_accuracy: 0.8337 - val_loss: 1.5137 - learning_rate: 1.0000e-04
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 24s 129ms/step - accuracy: 0.8408 - loss: 1.3784 - val_accuracy: 0.8337 - val_loss: 1.2202 - learning_rate: 1.0000e-04
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.8445 - loss: 1.1121 - val_accuracy: 0.8438 - val_loss: 1.1600 - learning_rate: 1.0000e-04
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 131ms/step - accuracy: 0.8510 - loss: 1.0617 - val_accuracy: 0.8455 - val_loss: 1.1591 - learning_rate: 1.0000e-04
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 0.8536 - loss: 1.0322 - val_accuracy: 0.8453 - val_loss: 1.1512 - learning_rate: 1.0000e-04
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 133ms/step - accuracy: 0.8545 - loss: 1.0133 - val_accuracy: 0.8453 - val_loss: 1.1556 - learning_rate: 1.0000e-04
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━

In [26]:
test_data['input_tokenized'] = test_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
test_data['output_tokenized'] = test_data['new_output'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

test_input_sequences = pad_sequences(test_data['input_tokenized'], maxlen=max_len, padding='post')
test_output_sequences = pad_sequences(test_data['output_tokenized'], maxlen=max_len, padding='post')

test_output_padded = np.expand_dims(test_output_sequences, -1)


<ipython-input-26-941d46fc29b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['input_tokenized'] = test_data['new_input'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
<ipython-input-26-941d46fc29b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['output_tokenized'] = test_data['new_output'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [ ]:
sample_input = test_input_sequences[1:2]
predicted_output = model.predict(sample_input)

predicted_output_text = tokenizer.decode(np.argmax(predicted_output[0], axis=-1))

true_output_text = tokenizer.decode(test_output_sequences[0])

print("Predicted Output:", predicted_output_text)
print("True Output:", true_output_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Output: [CLS] you you put a americmeric. numericmericindmericmeric nu. f [PAD] [PAD] f f [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
True Output: [CLS] noneus is this blackdown or sun f [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# weighted class


In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy

pad_token_weight = 0.1
word_token_weight = 1.0

def weighted_sparse_categorical_crossentropy(y_true, y_pred):
    scce_loss = sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    weights = mask * word_token_weight + (1 - mask) * pad_token_weight

    weighted_loss = scce_loss * weights

    return tf.reduce_mean(weighted_loss)

model.compile(optimizer=optimizer, loss=weighted_sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(
    train_input_sequences,
    train_output_sequences,
    epochs=100,
    batch_size=32,
    validation_data=(val_input_sequences, val_output_sequences),
    callbacks=[reduce_lr]
)


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 66s 360ms/step - accuracy: 0.9544 - loss: 0.1640 - val_accuracy: 0.7969 - val_loss: 2.8167 - learning_rate: 1.0000e-04
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 42s 145ms/step - accuracy: 0.9546 - loss: 0.1615 - val_accuracy: 0.7962 - val_loss: 2.8388 - learning_rate: 1.0000e-04
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 147ms/step - accuracy: 0.9565 - loss: 0.1537 - val_accuracy: 0.7989 - val_loss: 2.8701 - learning_rate: 1.0000e-04
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 15s 142ms/step - accuracy: 0.9580 - loss: 0.1440 - val_accuracy: 0.7989 - val_loss: 2.9235 - learning_rate: 1.0000e-04
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 15s 139ms/step - accuracy: 0.9594 - loss: 0.1411 - val_accuracy: 0.7938 - val_loss: 2.8753 - learning_rate: 1.0000e-04
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 15s 143ms/step - accuracy: 0.9594 - loss: 0.1391 - val_accuracy: 0.7969 - val_loss: 2.9193 - learning_rate: 1.0000e-04
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
sample_input = test_input_sequences[17:18]
predicted_output = model.predict(sample_input)

predicted_output_text = tokenizer.decode(np.argmax(predicted_output[0], axis=-1))

true_output_text = tokenizer.decode(test_output_sequences[17])

print("Predicted Output:", predicted_output_text)
print("True Output:", true_output_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Output: [CLS] the found nus default search f nu [PAD] [PAD] f [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
True Output: [CLS] i do not what is up no matter what java plugin i use it take a really long time to load a java applet f [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

all_tokens = np.concatenate(train_data['input_tokenized'].values)
unique_tokens = np.unique(all_tokens)

class_weights = compute_class_weight('balanced', classes=unique_tokens, y=all_tokens)

pad_token_id = tokenizer.pad_token_id
class_weight_dict = {token: weight for token, weight in zip(unique_tokens, class_weights)}

if pad_token_id in class_weight_dict:
    class_weight_dict[pad_token_id] = 0.1


In [ ]:
class_weight_dict[pad_token_id] = 0.00001

class_weights_tensor = tf.constant([
    class_weight_dict.get(i, 1.0) for i in range(max(class_weight_dict.keys()) + 1)
], dtype=tf.float32)

def weighted_sparse_categorical_crossentropy(y_true, y_pred):
    scce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)

    weights = tf.gather(class_weights_tensor, tf.cast(y_true, tf.int32))  # Gather weights based on y_true values
    weighted_loss = scce * weights

    return weighted_loss

model.compile(optimizer=optimizer, loss=weighted_sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(
    train_input_sequences,
    train_output_sequences,
    validation_data=(val_input_sequences, val_output_sequences),
    epochs=10,
    batch_size=32
)


Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 67s 360ms/step - accuracy: 0.9742 - loss: 0.0195 - val_accuracy: 0.7654 - val_loss: 6.3385
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 140ms/step - accuracy: 0.9367 - loss: 0.0206 - val_accuracy: 0.7709 - val_loss: 6.7586
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 152ms/step - accuracy: 0.9455 - loss: 0.0176 - val_accuracy: 0.7710 - val_loss: 6.7073
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 143ms/step - accuracy: 0.9522 - loss: 0.0155 - val_accuracy: 0.7709 - val_loss: 6.8977
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 20s 138ms/step - accuracy: 0.9536 - loss: 0.0111 - val_accuracy: 0.7822 - val_loss: 6.9675
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 139ms/step - accuracy: 0.9556 - loss: 0.0103 - val_accuracy: 0.7813 - val_loss: 7.2031
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 149ms/step - accuracy: 0.9638 - loss: 0.0091 - val_accuracy: 0.7768 - val_loss: 7.0672
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 16s 149ms/step - accuracy: 0.9642 - loss: 0

In [ ]:
sample_input = test_input_sequences[17:18]
predicted_output = model.predict(sample_input)

predicted_output_text = tokenizer.decode(np.argmax(predicted_output[0], axis=-1))

true_output_text = tokenizer.decode(test_output_sequences[17])

print("Predicted Output:", predicted_output_text)
print("True Output:", true_output_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Output: [CLS] keep found found default nano searchioner namerso [SEP]. [PAD] f [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
True Output: [CLS] i do not what is up no matter what java plugin i use it take a really long time to load a java applet f [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# L2 + DROP OUT

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, l2_reg=1e-4):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu", kernel_regularizer=l2(l2_reg)),
            Dense(embed_dim, kernel_regularizer=l2(l2_reg)),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim, l2_reg=1e-4):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, embeddings_regularizer=l2(l2_reg))
        self.pos_emb = Embedding(input_dim=max_len, output_dim=embed_dim, embeddings_regularizer=l2(l2_reg))

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def transformer_model(vocab_size, max_len, embed_dim=64, num_heads=4, ff_dim=256, num_transformer_blocks=2, rate=0.1, l2_reg=1e-4):
    inputs = Input(shape=(max_len,))
    embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim, l2_reg)
    x = embedding_layer(inputs)

    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate, l2_reg)(x)

    x = Dense(ff_dim, activation="relu", kernel_regularizer=l2(l2_reg))(x)
    x = Dropout(rate)(x)
    outputs = Dense(vocab_size, activation="softmax", kernel_regularizer=l2(l2_reg))(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create a smaller model with L2 regularization
model = transformer_model(vocab_size=vocab_size, max_len=max_len)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.build(input_shape=(None, max_len))
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 100, 64)             │       1,926,400 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 100, 64)             │          99,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_1                  │ (None, 100, 64)             │          99,712 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100, 256)            │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 100, 30000)          │       7,710,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,852,464 (37.58 MB)

 Trainable params: 9,852,464 (37.58 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.001)
pad_token_weight = 0.1
word_token_weight = 1.0

def weighted_sparse_categorical_crossentropy(y_true, y_pred):
    scce_loss = sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    weights = mask * word_token_weight + (1 - mask) * pad_token_weight

    weighted_loss = scce_loss * weights

    return tf.reduce_mean(weighted_loss)

model.compile(optimizer=optimizer, loss=weighted_sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(
    train_input_sequences,
    train_output_sequences,
    epochs=100,
    batch_size=32,
    validation_data=(val_input_sequences, val_output_sequences),
    callbacks=[reduce_lr]
)


Epoch 1/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 88s 90ms/step - accuracy: 0.7495 - loss: 1.8479 - val_accuracy: 0.8388 - val_loss: 1.0534 - learning_rate: 1.0000e-04
Epoch 2/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 44s 62ms/step - accuracy: 0.8349 - loss: 1.0312 - val_accuracy: 0.8250 - val_loss: 1.0094 - learning_rate: 1.0000e-04
Epoch 3/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.8307 - loss: 0.9997 - val_accuracy: 0.8218 - val_loss: 0.9995 - learning_rate: 1.0000e-04
Epoch 4/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 38s 61ms/step - accuracy: 0.8309 - loss: 0.9883 - val_accuracy: 0.8356 - val_loss: 0.9899 - learning_rate: 1.0000e-04
Epoch 5/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 38s 61ms/step - accuracy: 0.8319 - loss: 0.9772 - val_accuracy: 0.8275 - val_loss: 0.9882 - learning_rate: 1.0000e-04
Epoch 6/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 42s 62ms/step - accuracy: 0.8327 - loss: 0.9707 - val_accuracy: 0.8300 - val_loss: 0.9841 - learning_rate: 1.0000e-04
Epoch 7/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 41s 6

In [32]:
sample_input = test_input_sequences[18:19]
predicted_output = model.predict(sample_input)

predicted_output_text = tokenizer.decode(np.argmax(predicted_output[0], axis=-1))

true_output_text = tokenizer.decode(test_output_sequences[19])

print("Predicted Output:", predicted_output_text)
print("True Output:", true_output_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Output: [CLS] i......mericmeric nu nu nu the the the the a the is is. f f [SEP] [SEP] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
True Output: [CLS] now get a pencil and paper f [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P